In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.constants import c, e, epsilon_0, m_e
plt.style.use('huntstone')
import prop_plasma as pp;
import v_x_interp as vx;

In [3]:
beam_params = {
    'N'       : int(1e6),
    'beamE'   : 10e9, 
    'eps_n0'  : 3.0e-6,
    'beta0'   : [0.25, 0.25], 
    'alpha0'  : [0.00, 0.00],
    'rms_z'  : 0.00,
    'rms_gb0' : 0.01,
    'path'    : '/media/keenan/Data_Storage/WARGSim/',
    'B_mag'   : 5.00
}
plasma_params = {
    'n0'    : 0.34,
    'L_ft'  : 0.4,
    'hw_up' : 0.0,
    'shape' : 'Gauss'
}
my_ebeam = pp.init_beam(beam_params, plasma_params['n0'])
pwfa0    = pp.init_plasma(beam_params, plasma_params)

beta factor = 43.83
Bmag = 21.92
init eps_nx =  2.9993413598927024e-06
init eps_ny =  3.0036524179275437e-06


In [5]:
name  = "Gain/"
nGoal = 500
dumpPeriod = int(len(pwfa0.dz) / nGoal)
nDumps = int(len(pwfa0.dz) / dumpPeriod)
print(nDumps)

699


In [4]:
pp.propagate(beam_params, plasma_params, dumpPeriod, name)

beta factor = 43.83
Bmag = 21.92
init eps_nx =  2.9982435361267406e-06
init eps_ny =  2.9993965019917192e-06


In [7]:
path   = "/media/keenan/Data_Storage/WARGSim/" + "noGain/"
nParts = int(1e6)
dz     = pwfa0.dz[0]
z      = np.linspace(0, pwfa0.L, nDumps)
x      = np.zeros((nDumps))
y      = np.zeros((nDumps))
#xp     = np.zeros((nDumps))
#yp     = np.zeros((nDumps))
gb     = np.zeros((nDumps))
t      = z / c;
t[0]   = 0;
for i in range(nDumps):
    if i%100 == 0 and i >=100:
        print(i)
    ptcls   = np.load(path + "_ptcls_" + str(i) + ".npy")
    x[i]  = ptcls[0,0]
    #xp[i] = ptcls[0, 1]
    y[i]  = ptcls[0, 2]
    #yp[i] = ptcls[0, 3]
    gb[i] = ptcls[0, 5] 
np.save("x_noGain.npy", x)
np.save("y_noGain.npy", y)
np.save("gb_noGain.npy", gb)

100
200
300
400
500
600
